# PRATICA GUIADA - Limpeza de dados.

#### O [Pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html) fornece um conjunto de métodos para trabalhar com [dados faltantes](https://towardsdatascience.com/data-cleaning-with-python-and-pandas-detecting-missing-values-3e9c6ebcf78b). Os métodos reconhecem como dados faltantes valores que podem vir de `Numpy` ou do `Python` nativo. 

In [1]:
import pandas as pd
import numpy as np

## Detecção de dados faltantes

#### Vamos construir uma `series` com valores faltantes. Usamos a função [`.Series()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html), que retornaum objeto `Ndarray` unidimensional com rótulos de eixo.

In [2]:
string_data = pd.Series(['aardvark', 
                         'artichoke', 
                         np.nan, 
                         'avocado'
                        ]
                       )
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

#### O método [`.isnull()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.isnull.html) retorna uma máscara booleana para a série que indica os dados faltantes. ele também reconhece o valor faltante do Python nativo.

In [3]:
string_data = pd.Series([None, 
                         'artichoke', 
                         np.nan, 
                         'avocado'
                        ]
                       )
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

#### Para encontrar os valores com dados faltantes, podemos filtrar a série usando [boolean indexing](https://mode.com/python-tutorial/python-filtering-with-boolean-indexes/). A seguir a filtrem os valores nulos, aqui usamos o método [`isnull()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.isnull.html), que detecta valores ausentes para um objeto.

In [4]:
print(string_data[string_data.isnull()])

0    None
2     NaN
dtype: object


In [5]:
type(string_data)

pandas.core.series.Series

#### E a seguir a filtrem os valores não nulos, aqui usamos o método [`notnull()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.notnull.html), que detecta valores existentes (não ausentes).

In [6]:
print(string_data[string_data.notnull()])

1    artichoke
3      avocado
dtype: object


#### Na hora de trabalhar com dataframes, podemos selecionar as linhas ou colunas que não contêm nenhum valor faltante. Vamos criar um dataframe a partir do método [`.randn()`](https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.random.randn.html), que retorne uma amostra (ou amostras) da distribuição [normal padrão](https://sphweb.bumc.bu.edu/otlt/mph-modules/bs/bs704_probability/bs704_probability9.html#:~:text=The%20standard%20normal%20distribution%20is,and%20standard%20deviation%20of%201.&text=For%20the%20standard%20normal%20distribution,standard%20deviations%20of%20the%20mean.).

In [7]:
df = pd.DataFrame(np.random.randn(7, 3))
df2 = df
df

,0,1,2
0,0.091442,0.762071,0.513832
1,0.243238,1.471765,1.080712
2,-1.868598,-0.712624,0.336558
3,0.552427,0.109905,-0.867731
4,0.374130,-0.720042,-1.554175
5,0.271487,0.465087,-0.926760
6,-0.844959,-2.139278,0.556639


#### Agora, geramos alguns dados faltantes, com o auxílio do método [`.iloc[]`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iloc.html), que realiza uma indexação puramente baseada em inteiros, para localização por posição.

In [8]:
df.iloc[:4, 1] = np.nan
df.iloc[2, 2] = None
df

,0,1,2
0,0.091442,NaN,0.513832
1,0.243238,NaN,1.080712
2,-1.868598,NaN,NaN
3,0.552427,NaN,-0.867731
4,0.374130,-0.720042,-1.554175
5,0.271487,0.465087,-0.926760
6,-0.844959,-2.139278,0.556639


#### É possível remover as linhas que apresentam `NaN` com o auxílio do método [`.dropna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html), que remove valores faltantes.

In [9]:
df.dropna(axis = 0)

,0,1,2
4,0.374130,-0.720042,-1.554175
5,0.271487,0.465087,-0.926760
6,-0.844959,-2.139278,0.556639


#### E é possível remover as colunas que apresentam `NaN`, repare no parâmetro [`axis`](https://towardsdatascience.com/understanding-axes-and-dimensions-numpy-pandas-606407a5f950).

In [10]:
df.dropna(axis = 1)

,0
0,0.091442
1,0.243238
2,-1.868598
3,0.552427
4,0.374130
5,0.271487
6,-0.844959


#### É possível definir um critério limitado para realizar o drop. Por exemplo, realizar a remoção das linhas que são `NaN` na coluna $2$.

In [11]:
df.dropna(axis = 0, subset = [2])

,0,1,2
0,0.091442,NaN,0.513832
1,0.243238,NaN,1.080712
3,0.552427,NaN,-0.867731
4,0.374130,-0.720042,-1.554175
5,0.271487,0.465087,-0.926760
6,-0.844959,-2.139278,0.556639


#### ou realizar a remoção das linhas que são `NaN` na coluna $3, 4, 5$ e $6$.

In [12]:
df.dropna(axis = 1, 
          subset = [3, 4, 5, 6]
         )

,0,2
0,0.091442,0.513832
1,0.243238,1.080712
2,-1.868598,NaN
3,0.552427,-0.867731
4,0.374130,-1.554175
5,0.271487,-0.926760
6,-0.844959,0.556639


## Métodos de imputação, preenchimento de dados faltantes.

#### Para discutir o preenchimento dos dados faltantes, vamos primeiro renomear as colunas com o método [`.columns()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.columns.html), que retorna os rótulos das colunas do DataFrame. 

In [13]:
df.columns = ['col1', 
              'col2', 
              'col3'
             ]
df

,col1,col2,col3
0,0.091442,NaN,0.513832
1,0.243238,NaN,1.080712
2,-1.868598,NaN,NaN
3,0.552427,NaN,-0.867731
4,0.374130,-0.720042,-1.554175
5,0.271487,0.465087,-0.926760
6,-0.844959,-2.139278,0.556639


#### Podemos optar pelo preenchimento com um escalar, este método retorna um objeto novo. Para alterar dataframe diretamente o parâmetro `inplace = True` é utilizado em [`.fillna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html).

In [18]:
df.fillna(0)

,col1,col2,col3
0,0.091442,0.000000,0.513832
1,0.243238,0.000000,1.080712
2,-1.868598,0.000000,0.000000
3,0.552427,0.000000,-0.867731
4,0.374130,-0.720042,-1.554175
5,0.271487,0.465087,-0.926760
6,-0.844959,-2.139278,0.556639


In [19]:
df

,col1,col2,col3
0,0.091442,0.000000,0.513832
1,0.243238,0.000000,1.080712
2,-1.868598,0.000000,0.000000
3,0.552427,0.000000,-0.867731
4,0.374130,-0.720042,-1.554175
5,0.271487,0.465087,-0.926760
6,-0.844959,-2.139278,0.556639


In [20]:
df.fillna(0, inplace = True)
df

,col1,col2,col3
0,0.091442,0.000000,0.513832
1,0.243238,0.000000,1.080712
2,-1.868598,0.000000,0.000000
3,0.552427,0.000000,-0.867731
4,0.374130,-0.720042,-1.554175
5,0.271487,0.465087,-0.926760
6,-0.844959,-2.139278,0.556639


#### O [preenchimento](https://towardsdatascience.com/missing-data-and-imputation-89e9889268c8) dos dados faltantes pode ser feito com um [dicionário](https://pandas.pydata.org/pandas-docs/version/1.0.0/user_guide/missing_data.html).

In [21]:
df = pd.DataFrame(np.random.randn(7, 3), 
                  columns = ['col1', 
                             'col2', 
                             'col3']
                 )

df.iloc[1:4, 1] = np.nan
df.iloc[:2, 2] = np.nan
df

,col1,col2,col3
0,-1.365294,-0.240053,NaN
1,-0.048600,NaN,NaN
2,0.832215,NaN,0.375619
3,0.066605,NaN,0.433648
4,1.551322,-0.266831,0.479088
5,0.051287,0.839936,0.335411
6,-0.780886,0.495170,-0.286230


#### Removemos alguns valores, substituindo-os por valroes `NaN` e agora podemos preenchê-los novamente com um dicionário aplicado ao método `.fillna()`.

In [29]:
df.fillna({'col2': 0.5, 'col3': -1})

,col1,col2,col3
0,-1.365294,-0.240053,-1.000000
1,-0.048600,0.500000,-1.000000
2,0.832215,0.500000,0.375619
3,0.066605,0.500000,0.433648
4,1.551322,-0.266831,0.479088
5,0.051287,0.839936,0.335411
6,-0.780886,0.495170,-0.286230


#### Para preencher com base nos últimos valores válidos, é possível usar a função [`.fillna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html), com o parâmetro method = 'ffill', que propaga a última observação válida para o próximo `backfill` válido.

In [30]:
df.fillna(method = 'ffill') 

,col1,col2,col3
0,-1.365294,-0.240053,NaN
1,-0.048600,-0.240053,NaN
2,0.832215,-0.240053,0.375619
3,0.066605,-0.240053,0.433648
4,1.551322,-0.266831,0.479088
5,0.051287,0.839936,0.335411
6,-0.780886,0.495170,-0.286230


#### A função função [`.fillna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html) também apresenta o parâmetro method = 'bfill', que usa a próxima observação válida para preencher a lacuna.

In [31]:
df.fillna(method = 'bfill') 

,col1,col2,col3
0,-1.365294,-0.240053,0.375619
1,-0.048600,-0.266831,0.375619
2,0.832215,-0.266831,0.375619
3,0.066605,-0.266831,0.433648
4,1.551322,-0.266831,0.479088
5,0.051287,0.839936,0.335411
6,-0.780886,0.495170,-0.286230


### O [preenchimento](https://towardsdatascience.com/a-comprehensive-guide-to-data-imputation-e82eadc22609) com a média e a média [condicionada](https://towardsdatascience.com/ds-in-the-real-world-5f77800aff78)

#### O método `.fillna()` também [aceita](https://towardsdatascience.com/data-imputation-to-improve-model-performance-c4eb2a9954ad) um novo dataframe com índices que coincidam com os valores faltantes. 

In [36]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6), 
                   'data2': np.random.rand(6)}, 
                  columns = ['key', 'data1','data2'] 
                 )
df

,key,data1,data2
0,A,0,0.993944
1,B,1,0.396002
2,C,2,0.278733
3,A,3,0.498762
4,B,4,0.893256
5,C,5,0.401799


In [37]:
df.iloc[2:3, 1] = np.nan
df.iloc[3:4, 2] = np.nan
df

,key,data1,data2
0,A,0.0,0.993944
1,B,1.0,0.396002
2,C,NaN,0.278733
3,A,3.0,NaN
4,B,4.0,0.893256
5,C,5.0,0.401799


In [40]:
print(df['data1'].mean())
print(df['data2'].mean())

2.6
0.5927469200303925


In [44]:
df.fillna(df.mean())

,key,data1,data2
0,A,0.0,0.993944
1,B,1.0,0.396002
2,C,2.6,0.278733
3,A,3.0,0.592747
4,B,4.0,0.893256
5,C,5.0,0.401799


#### É possível calcular as médias do dataframe usando os métodos [`.groupby()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html?highlight=groupby#pandas.DataFrame.groupby) e [`.transform()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.transform.html), agrupando o dataframe pela coluna `'key'`.

In [45]:
df.groupby(by = 'key').transform('mean')

,data1,data2
0,1.5,0.993944
1,2.5,0.644629
2,5.0,0.340266
3,1.5,0.993944
4,2.5,0.644629
5,5.0,0.340266


#### Também é possível realizar a utilizando o método `.fillna()`.

In [46]:
df.fillna(df.groupby('key').transform('mean'))

,key,data1,data2
0,A,0.0,0.993944
1,B,1.0,0.396002
2,C,5.0,0.278733
3,A,3.0,0.993944
4,B,4.0,0.893256
5,C,5.0,0.401799


## Tidy Data.

#### Vamos trabalhar com alguns exemplos de `messy data` presentes no [trabalho original](https://vita.had.co.nz/papers/tidy-data.pdf) de [Hadley Wickham](https://en.wikipedia.org/wiki/Hadley_Wickham). 

#### A ideia é esbarrarmos com conjuntos de dados como eles poderiam existir no mundo real e passá-los para um formato com que as ferramentas padrão de mineração de dados e visualização possam trabalhar melhor, conforme as regras de ['tidy data'](https://towardsdatascience.com/whats-tidy-data-how-to-organize-messy-datasets-in-python-with-melt-and-pivotable-functions-5d52daa996c9).

#### Vamos trabalhar com alguns tipos de conjuntos de dados desordenados, sobre a relação entre renda e religião nos EUA, coletados pelo ['Pew Research Center'](https://www.pewresearch.org/).

#### Os nomes de colunas são valores, não variáveis:

In [48]:
import pandas as pd
import datetime
from os import listdir
from os.path import isfile, join
import glob
import re


df = pd.read_csv("pew-raw.csv")
df

,religion,<$10k,$10-20k,$20-30k,$30-40,$40-50,$50-75
0,Agnostic,27,34,60,81,76,137
1,Atheist,12,27,37,52,35,70
2,Buddhist,27,21,30,34,33,58
3,Catholic,418,617,732,670,638,1116
4,Dont Know / refused,15,14,15,11,10,35
5,Evangelical Prot,575,869,1064,982,881,1486
6,Hindu,1,9,7,9,11,34
7,Historically Black Prot,228,224,236,238,197,223
8,Jehovahs Witness,20,27,24,24,21,30
9,Jewish,19,19,25,25,30,95


#### Para reorganizar o conjunto de dados, utilizamos o método [`.melt()`](https://pandas.pydata.org/docs/reference/api/pandas.melt.html). que desvincula (`unpivot`) um `DataFrame` do formato largo para o longo, opcionalmente deixando o conjunto de identificadores.

#### Nos parâmetros, indicamos que a variável que vamos conservar é `'religion'` (é possível conservar mais variáveis). E que com o restante das colunas vamos construir uma nova variável onde cada coluna seja uma categoria.

In [51]:
df_ordenado = pd.melt(df, 
                      ["religion"], 
                      var_name = "income", 
                      value_name = "freq"
                     )

df_ordenado.head(10)

,religion,income,freq
0,Agnostic,<$10k,27
1,Atheist,<$10k,12
2,Buddhist,<$10k,27
3,Catholic,<$10k,418
4,Dont Know / refused,<$10k,15
5,Evangelical Prot,<$10k,575
6,Hindu,<$10k,1
7,Historically Black Prot,<$10k,228
8,Jehovahs Witness,<$10k,20
9,Jewish,<$10k,19


-  Para desfazer pode utilizar a função [`.pivot_table()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html), que cria uma tabela dinâmica no estilo planilha como um `DataFrame`.

-  Como a `pivot_table` usa o `index` do dataframe de saída como variável para a linha, é possível retornar os dados do índice para uma coluna usando a funcao [`.reset_index()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reset_index.html), redefine o índice do DataFrame e use o padrão. Se o DataFrame tiver um MultiIndex, este método pode remover um ou mais níveis.

In [52]:
df_ordenado.pivot_table(index = 'religion', 
                        columns = 'income', 
                        values = 'freq').reset_index()

income,religion,$10-20k,$20-30k,$30-40,$40-50,$50-75,<$10k
0,Agnostic,34,60,81,76,137,27
1,Atheist,27,37,52,35,70,12
2,Buddhist,21,30,34,33,58,27
3,Catholic,617,732,670,638,1116,418
4,Dont Know / refused,14,15,11,10,35,15
5,Evangelical Prot,869,1064,982,881,1486,575
6,Hindu,9,7,9,11,34,1
7,Historically Black Prot,224,236,238,197,223,228
8,Jehovahs Witness,27,24,24,21,30,20
9,Jewish,19,25,25,30,95,19


#### Mais de um valor na mesma coluna

#### A seguir, vamos usar dados da OMS. O conjunto de dados é composto pela quantidade de casos de tuberculose observados por país, ano, sexo e idade.  

In [57]:
df = pd.read_csv("tb-raw.csv")
df

,country,year,m014,m1524,m2534,m3544,m4554,m5564,m65,mu,f014
0,AD,2000,0.0,0.0,1.0,0.0,0,0,0.0,NaN,NaN
1,AE,2000,2.0,4.0,4.0,6.0,5,12,10.0,NaN,3.0
2,AF,2000,52.0,228.0,183.0,149.0,129,94,80.0,NaN,93.0
3,AG,2000,0.0,0.0,0.0,0.0,0,0,1.0,NaN,1.0
4,AL,2000,2.0,19.0,21.0,14.0,24,19,16.0,NaN,3.0
5,AM,2000,2.0,152.0,130.0,131.0,63,26,21.0,NaN,1.0
6,AN,2000,0.0,0.0,1.0,2.0,0,0,0.0,NaN,0.0
7,AO,2000,186.0,999.0,1003.0,912.0,482,312,194.0,NaN,247.0
8,AR,2000,97.0,278.0,594.0,402.0,419,368,330.0,NaN,121.0
9,AS,2000,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN


#### Para ordenar este conjunto de dados, vamos extrair os valores de sexo e idade, a fim de organizá-los em uma única coluna. Depois, vamos criar três colunas com base no conteúdo: sexo, idade_de e idade_até.

In [58]:
df = pd.melt(df, 
             id_vars = ["country", "year"], 
             var_name = "sex_and_age", 
             value_name = "cases", 
            )
df.head()

,country,year,sex_and_age,cases
0,AD,2000,m014,0.0
1,AE,2000,m014,2.0
2,AF,2000,m014,52.0
3,AG,2000,m014,0.0
4,AL,2000,m014,2.0


#### Vamos realizar a [extração de variáveis](https://towardsdatascience.com/regular-expressions-in-python-a212b1c73d7f), com a ajudar das [expressões regulares](https://docs.python.org/3/library/re.html) e da função [`.str.extract()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.extract.html), que extrai grupos de captura no `regex pat` (padrão de expressão regular com captura de grupos), como colunas em um `DataFrame`.

#### Pedimos à função que ela divida o valor que recebe em três partes:

- (`\D`): Uma única letra ou caractere não numérico.
- (`\d+`): Um ou mais números (para dar conta de "idade de").
- (`\d{2}`): Dois dígitos.

In [59]:
tmp_df = df["sex_and_age"].str.extract("(\D)(\d+)(\d{2})", 
                                       expand = False
                                      )
tmp_df

,0,1,2
0,m,0,14
1,m,0,14
2,m,0,14
3,m,0,14
4,m,0,14
...,...,...,...
85,f,0,14
86,f,0,14
87,f,0,14
88,f,0,14


In [60]:
pd.Series(['m014']).str.extract("(\D)(\d+)(\d{2})", 
                                expand = False
                               )

,0,1,2
0,m,0,14


#### Atribuímos os nomes `"sex"`, `"age_lower"` e  `"age_upper"` às colunas do dataframe `tmp_df`.

In [61]:
tmp_df.columns = ["sex", "age_lower", "age_upper"]
tmp_df

,sex,age_lower,age_upper
0,m,0,14
1,m,0,14
2,m,0,14
3,m,0,14
4,m,0,14
...,...,...,...
85,f,0,14
86,f,0,14
87,f,0,14
88,f,0,14


#### Criamos a coluna idade com base em `"age_lower"` e `"age_upper"`.

In [62]:
tmp_df["age"] = tmp_df["age_lower"] + "-" + tmp_df["age_upper"]
tmp_df

,sex,age_lower,age_upper,age
0,m,0,14,0-14
1,m,0,14,0-14
2,m,0,14,0-14
3,m,0,14,0-14
4,m,0,14,0-14
...,...,...,...,...
85,f,0,14,0-14
86,f,0,14,0-14
87,f,0,14,0-14
88,f,0,14,0-14


In [63]:
df

,country,year,sex_and_age,cases
0,AD,2000,m014,0.0
1,AE,2000,m014,2.0
2,AF,2000,m014,52.0
3,AG,2000,m014,0.0
4,AL,2000,m014,2.0
...,...,...,...,...
85,AM,2000,f014,1.0
86,AN,2000,f014,0.0
87,AO,2000,f014,247.0
88,AR,2000,f014,121.0


#### Unimos os dos conjuntos de dados com a ajuda da função [`.concat()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html), que concatene objetos pandas ao longo de um eixo específico com lógica de conjunto opcional ao longo dos outros eixos.

In [64]:
df = pd.concat([df, tmp_df], 
               axis = 1
              )
df.head()

,country,year,sex_and_age,cases,sex,age_lower,age_upper,age
0,AD,2000,m014,0.0,m,0,14,0-14
1,AE,2000,m014,2.0,m,0,14,0-14
2,AF,2000,m014,52.0,m,0,14,0-14
3,AG,2000,m014,0.0,m,0,14,0-14
4,AL,2000,m014,2.0,m,0,14,0-14


In [65]:
df["age"].value_counts()

0-14     20
45-54    10
55-64    10
35-44    10
15-24    10
25-34    10
Name: age, dtype: int64

#### Conferimos a presença de valores faltantes.

In [66]:
np.sum(df.isnull())

country         0
year            0
sex_and_age     0
cases          17
sex            20
age_lower      20
age_upper      20
age            20
dtype: int64

#### Analisando os casos faltantes, observamos que a expressão regular não funcionou para `'m65'`ou `'mu'`.

In [75]:
#df['sex_and_age'].unique()
df.loc[df['age'].isnull(), ]

,country,year,sex_and_age,cases,sex,age_lower,age_upper,age
60,AD,2000,m65,0.0,NaN,NaN,NaN,NaN
61,AE,2000,m65,10.0,NaN,NaN,NaN,NaN
62,AF,2000,m65,80.0,NaN,NaN,NaN,NaN
63,AG,2000,m65,1.0,NaN,NaN,NaN,NaN
64,AL,2000,m65,16.0,NaN,NaN,NaN,NaN
65,AM,2000,m65,21.0,NaN,NaN,NaN,NaN
66,AN,2000,m65,0.0,NaN,NaN,NaN,NaN
67,AO,2000,m65,194.0,NaN,NaN,NaN,NaN
68,AR,2000,m65,330.0,NaN,NaN,NaN,NaN
69,AS,2000,m65,NaN,NaN,NaN,NaN,NaN


In [76]:
df.loc[df['sex_and_age'] == 'm65', 'age'] = '65 or more'
df.loc[df['sex_and_age'] == 'm65', 'sex'] = 'm'
df.loc[df['age'].isnull(), ]
#df.tail()

,country,year,sex_and_age,cases,sex,age_lower,age_upper,age
70,AD,2000,mu,NaN,NaN,NaN,NaN,NaN
71,AE,2000,mu,NaN,NaN,NaN,NaN,NaN
72,AF,2000,mu,NaN,NaN,NaN,NaN,NaN
73,AG,2000,mu,NaN,NaN,NaN,NaN,NaN
74,AL,2000,mu,NaN,NaN,NaN,NaN,NaN
75,AM,2000,mu,NaN,NaN,NaN,NaN,NaN
76,AN,2000,mu,NaN,NaN,NaN,NaN,NaN
77,AO,2000,mu,NaN,NaN,NaN,NaN,NaN
78,AR,2000,mu,NaN,NaN,NaN,NaN,NaN
79,AS,2000,mu,NaN,NaN,NaN,NaN,NaN


#### Checamos se o número de nulos diminuiu.

In [77]:
np.sum(df.isnull())

country         0
year            0
sex_and_age     0
cases          17
sex            10
age_lower      20
age_upper      20
age            10
dtype: int64

#### Excluímos as colunas sobrantes.

In [78]:
df = df.drop(['sex_and_age', 
              "age_lower", 
              "age_upper"], 
             axis = 1
            )
df.head()

,country,year,cases,sex,age
0,AD,2000,0.0,m,0-14
1,AE,2000,2.0,m,0-14
2,AF,2000,52.0,m,0-14
3,AG,2000,0.0,m,0-14
4,AL,2000,2.0,m,0-14


#### Como as pessoas com idade indefinida não apresentam nenhum caso, é correto eliminar esses faltantes com dropna.

In [79]:
df = df.dropna()
df = df.sort_values(["country", 
                     "year", 
                     "sex", 
                     "age"]
                   )
df.head(10)

,country,year,cases,sex,age
0,AD,2000,0.0,m,0-14
10,AD,2000,0.0,m,15-24
20,AD,2000,1.0,m,25-34
30,AD,2000,0.0,m,35-44
40,AD,2000,0.0,m,45-54
50,AD,2000,0.0,m,55-64
60,AD,2000,0.0,m,65 or more
81,AE,2000,3.0,f,0-14
1,AE,2000,2.0,m,0-14
11,AE,2000,4.0,m,15-24


####  <span style = "color:red">Parei Aqui.</span>

## Ferramentas para a limpeza e manipulação de dados

#### O Pandas tem um conjunto de métodos que permitem operar sobre os elementos de um Dataframe ou uma Series. Para aplicar a lógica desejada, podemos definir funções com nome ou utilizar expressões lambda que depois não podem ser reutilizadas

* [pd.DataFrame.apply()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html): Opera sobre linhas ou colunas completas.
* [pd.DataFrame.applymap()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.applymap.html): Opera sobre cada um dos elementos do Dataframe.
* [pd.Series.apply()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.apply.html): Opera sobre cada um dos elementos da Série. 
* [pd.Series.map()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.map.html): Opera sobre cada um dos elementos da Serie, muito parecido com `pd.Series.apply()`. 

### Função [`.apply()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html).

#### A função apply do Pandas permite realizar [operações](https://towardsdatascience.com/apply-function-to-pandas-dataframe-rows-76df74165ee4) vetorizadas sobre os conjuntos de dados tanto linha por linha quanto coluna por coluna.

In [81]:
import pandas as pd
import numpy as np
df = pd.DataFrame(np.random.randn(5, 4), 
                  columns = ['a', 'b', 'c', 'd']
                 )
df

,a,b,c,d
0,1.399713,-0.881477,-1.215143,-0.506040
1,-0.330280,0.567447,-0.463582,-1.128178
2,-0.056459,-0.038635,-0.432246,1.542079
3,1.248648,1.896588,-0.095898,-0.393411
4,-0.576892,-0.096164,0.473868,0.481953


#### Utilizamos `.apply()` para encontrar a raiz quadrada dos elementos de cada coluna. A quantidade `NaN` significa ['Not a Number'](https://riptutorial.com/python/example/3973/infinity-and-nan---not-a-number--) e é o valor atribuído a operações inválidas, como a raiz de um número negativo. 

In [83]:
df.apply(np.sqrt)

,a,b,c,d
0,1.183095,NaN,NaN,NaN
1,NaN,0.753291,NaN,NaN
2,NaN,NaN,NaN,1.241805
3,1.117429,1.377167,NaN,NaN
4,NaN,NaN,0.68838,0.694229


#### O parâmetro `axis = 0` faz referência às colunas, esse é o eixo reduzido.

In [85]:
df.apply(np.mean, 
         axis = 0
        )

a    0.336946
b    0.289552
c   -0.346600
d   -0.000719
dtype: float64

#### Procuramos a média de todas as linhas, o parâmetro `axis = 1` indica que a função é aplicada a cada linha. 

#### Observe que o `.apply()` anterior não alterou o conjunto de dados, mas criou uma cópia e depois a alterou. O conjunto de dados original conserva o mesmo valor.

In [86]:
df.apply(np.mean, 
         axis = 1
        )

0   -0.300737
1   -0.338648
2    0.253685
3    0.663981
4    0.070691
dtype: float64

#### A função [`np.mean()`](https://numpy.org/doc/stable/reference/generated/numpy.mean.html) é definida em `numpy`, mas podemos querer aplicar uma função totalmente própria para, por exemplo, criar uma nova coluna que seja a adição entre as séries `a` e `d`. Isso pode ser feito com expressões [`lambda`](https://python-reference.readthedocs.io/en/latest/docs/operators/lambda.html).

#### Vejaamos como as expressões [`lambda`](https://towardsdatascience.com/what-are-lambda-functions-in-python-and-why-you-should-start-using-them-right-now-75ab85655dc6) funcionam.

In [90]:
df.apply(lambda x: print(type(x),'\n', x), 
         axis = 0
        )
#print(df)

<class 'pandas.core.series.Series'> 
 0    1.399713
1   -0.330280
2   -0.056459
3    1.248648
4   -0.576892
Name: a, dtype: float64
<class 'pandas.core.series.Series'> 
 0   -0.881477
1    0.567447
2   -0.038635
3    1.896588
4   -0.096164
Name: b, dtype: float64
<class 'pandas.core.series.Series'> 
 0   -1.215143
1   -0.463582
2   -0.432246
3   -0.095898
4    0.473868
Name: c, dtype: float64
<class 'pandas.core.series.Series'> 
 0   -0.506040
1   -1.128178
2    1.542079
3   -0.393411
4    0.481953
Name: d, dtype: float64


a    None
b    None
c    None
d    None
dtype: object

#### As funções `.map()`, `.apply()` e `.applymap()` são muito convenientes para usar na [limpeza de dados](https://medium.com/@boshengwu1994/introduction-to-apply-map-applymap-in-pandas-47706b44e59d). 

#### Por exemplo, vamos supor que queremos tirar todos os acentos e caracteres próprios do espanhol de todas as strings de um Dataframe.

#### Além disso, queremos converter todas as letras para minúscula.

In [92]:
data = pd.DataFrame({'nome': ['Tomás', 
                              'Carla', 
                              'Paula'], 
                     'sobrenome': ['Torres', 
                                   'López', 
                                   'Núñez']
                    }, 
                    columns = ['nome', 'sobrenome']
                   )

data

,nome,sobrenome
0,Tomás,Torres
1,Carla,López
2,Paula,Núñez


#### Vamos instalar e importar a biblioteca [`unidecode 1.1.1`](https://pypi.org/project/Unidecode/), que 
toma dados [`Unicode`](https://en.wikipedia.org/wiki/Unicode) e tenta representá-los em caracteres [`ASCII`](https://en.wikipedia.org/wiki/ASCII) (American Standard Code for Information Interchange).

In [93]:
#! conda install unidecode 
import unidecode
def quitar_caracteres(entrada):
    return str.lower(unidecode.unidecode(entrada))

In [94]:
data['sobrenome'].apply(quitar_caracteres)

0    torres
1     lopez
2     nunez
Name: sobrenome, dtype: object

In [95]:
data['sobrenome'].transform(quitar_caracteres)

0    torres
1     lopez
2     nunez
Name: sobrenome, dtype: object

In [96]:
data['sobrenome'].map(quitar_caracteres)

0    torres
1     lopez
2     nunez
Name: sobrenome, dtype: object

In [97]:
data.applymap(quitar_caracteres)

,nome,sobrenome
0,tomas,torres
1,carla,lopez
2,paula,nunez
